In [1]:
from jitcdde import jitcdde, t
from parameters import *
import numpy as np
from variables import *
from heat_transfer import *
from solver.objects import Node, System

In [2]:
# ARE system        
ARE = System()

# instantiate nodes
# core nodes 
c_f1 = Node(m = m_f_c/2, W = W_f, y0 = T0_c_f1)
c_f2 = Node(m = m_f_c/2, W = W_f, y0 = T0_c_f2)
c_t1 = Node(y0 = T0_c_t1)
c_c1 = Node(m = m_c_c/2, W = W_c, y0 = T0_c_c1)
c_c2 = Node(m = m_c_c/2, W = W_c, y0 = T0_c_c2) 
c_m1 = Node(y0 = T0_c_m+50)
n = Node(y0 = n_frac0)
C1 = Node(y0 = C0[0])
rho = Node(y0 = 0.0)

ARE.addNodes([c_f1,c_f2,c_t1,c_c1,c_c2,c_m1,n,C1,rho])

In [3]:
# define dynamics
# core nodes
c_f1.set_dTdt_bulkFlow(source = 500.00) 
c_f1.set_dTdt_internal(source = n.y, k = k_f1*P/mcp_f_c)
c_f1.set_dTdt_convective(source = [c_t1.y], hA_mcp = [hA_ft_c/mcp_f_c])

c_f2.set_dTdt_bulkFlow(source = c_f1.y) 
c_f2.set_dTdt_internal(source = n.y, k = k_f2*P/mcp_f_c)
c_f2.set_dTdt_convective(source = [c_t1.y], hA_mcp = [hA_ft_c/mcp_f_c])

c_t1.set_dTdt_convective(source = [c_f1.y, c_f2.y, c_c1.y, c_c2.y], 
                         hA_mcp = [hA_ft_c/mcp_t_c, hA_ft_c/mcp_t_c, hA_tc_c/mcp_t_c,hA_tc_c/mcp_t_c])

c_c1.set_dTdt_bulkFlow(source = 500.00)
c_c1.set_dTdt_convective(source = [c_t1.y,c_m1.y], hA_mcp = [hA_tc_c/mcp_c_c,hA_mc_c/mcp_c_c])

c_c2.set_dTdt_bulkFlow(source = c_c1.y)
c_c2.set_dTdt_convective(source = [c_t1.y,c_m1.y], hA_mcp = [hA_tc_c/mcp_c_c,hA_mc_c/mcp_c_c])

c_m1.set_dTdt_internal(source = n.y, k = k_m*P/mcp_m_c)
c_m1.set_dTdt_convective(source = [c_c1.y, c_c2.y], hA_mcp = [hA_mc_c/mcp_m_c]*2)

n.set_dndt(rho.y, beta_t, Lam, [lam[0]], [C1.y])
C1.set_dcdt(n.y, beta[0], Lam, lam[0], tau_c, tau_l)
rho.set_drdt([c_f1.dydt,c_f2.dydt,c_m1.dydt,c_c1.dydt,c_c2.dydt],[a_f/2,a_f/2,a_b,a_c/2,a_c/2])

# instantiate jitcdde object
DDE = jitcdde([n.dydt() for n in ARE.nodes])

# set initial conditions
DDE.constant_past([n.y0 for n in ARE.nodes])

In [4]:
DDE.get_state()

Generating, compiling, and loading C code.
Using default integration parameters.


[(-1.0,
  array([9.27038889e+02, 1.10092778e+03, 9.31761111e+02, 9.36483333e+02,
         9.97038889e+02, 1.02759444e+03, 1.00000000e+00, 1.56128208e+01,
         0.00000000e+00]),
  array([0., 0., 0., 0., 0., 0., 0., 0., 0.])),
 (0.0,
  array([9.27038889e+02, 1.10092778e+03, 9.31761111e+02, 9.36483333e+02,
         9.97038889e+02, 1.02759444e+03, 1.00000000e+00, 1.56128208e+01,
         0.00000000e+00]),
  array([0., 0., 0., 0., 0., 0., 0., 0., 0.]))]

In [5]:
c_c1.dydt()

0.0770480200560468*(500.0 - current_y(3)) + 0.0598891073963077*(current_y(2) - current_y(3)) + 0.00377850861311447*(-current_y(3) + current_y(5))

In [6]:
T = np.arange(0.0,250,0.01)
sol_jit = []
for t_x in T:
    sol_jit.append(DDE.integrate(t_x))

print(sol_jit[-1][6])

8.348776325023172


/mnt/envs/thesis_env/lib/python3.9/site-packages/jitcdde/_jitcdde.py:795: UserWarning: You did not explicitly handle initial discontinuities. Proceed only if you know what you are doing. This is only fine if you somehow chose your initial past such that the derivative of the last anchor complies with the DDE. In this case, you can set the attribute `initial_discontinuities_handled` to `True` to suppress this warning. See https://jitcdde.rtfd.io/#discontinuities for details.
  warn("You did not explicitly handle initial discontinuities. Proceed only if you know what you are doing. This is only fine if you somehow chose your initial past such that the derivative of the last anchor complies with the DDE. In this case, you can set the attribute `initial_discontinuities_handled` to `True` to suppress this warning. See https://jitcdde.rtfd.io/#discontinuities for details.")
/mnt/envs/thesis_env/lib/python3.9/site-packages/jitcdde/_jitcdde.py:792: UserWarning: The target time is smaller than 